In [1]:
from multiprocessing import Process, Queue, current_process, freeze_support
from random import choice

In [3]:
def worker(input, output):
    for func, args in iter(input.get, 'STOP'):
        result = calculate(func, args)
        output.put(result)

In [4]:
def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % \
        (current_process().name, func.__name__, args, result)

In [5]:
def mul(x,y):
    return x*y

def pow(x,y):
    return x^y

def minus(x,y):
    return x-y

def plus(x,y):
    return x+y


methods = [mul, pow, minus, plus]

In [6]:
NUMBER_OF_PROCESSES = 4

In [7]:
TASKS1 = [(choice(methods), i) for i in zip(range(20), range(20,0,-1))]

# Create queues
task_queue = Queue()
done_queue = Queue()

# Submit tasks
for task in TASKS1:
    task_queue.put(task)

In [8]:
# Start worker processes
for i in range(NUMBER_OF_PROCESSES):
    Process(target=worker, args=(task_queue, done_queue)).start()

In [9]:
# Get and print results
print('Unordered results:')
for i in range(len(TASKS1)):
    print('\t', done_queue.get())

Unordered results:


KeyboardInterrupt: 

In [ ]:
TASKS2 = [(choice(methods), i) for i in zip(range(20,40), range(20,0,-1))]
# Add more tasks using `put()`
for task in TASKS2:
    task_queue.put(task)

In [ ]:
# Get and print some more results
for i in range(len(TASKS2)):
    print('\t', done_queue.get())

In [ ]:
# Tell child processes to stop
for i in range(NUMBER_OF_PROCESSES):
    task_queue.put('STOP')

# A little bit of parallel computation.

Let's first check that we can get it to [work](https://ipyparallel.readthedocs.io/en/stable/index.html)

In [1]:
try:
    import ipyparallel as ipp
except ImportError:
    %pip install ipyparallel
    import ipyparallel as ipp
import time

In [2]:

task_durations = [1]*25

with ipp.Cluster() as rc:  # remote computer
    view = rc.load_balanced_view()

    async_result = view.map_async(time.sleep, task_durations)
    async_result.wait_interactive()
    result = async_result.get()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

sleep:   0%|          | 0/25 [00:00<?, ?tasks/s]

Stopping engine(s): 1640898292
engine set stopped 1640898292: {'engines': {'0': {'exit_code': 1, 'pid': 4928, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 444, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 15512, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 12388, 'identifier': '3'}, '4': {'exit_code': 1, 'pid': 13464, 'identifier': '4'}, '5': {'exit_code': 1, 'pid': 15680, 'identifier': '5'}, '6': {'exit_code': 1, 'pid': 3076, 'identifier': '6'}, '7': {'exit_code': 1, 'pid': 2324, 'identifier': '7'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 76, 'identifier': 'ipcontroller-1640898291-4w5z-8492'}


Now let's create some function that will do a bit of work

In [3]:
def largest_prime_factor(n):
    p = n
    i = 2
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
    return p, n

We can set the benchmark using the main python process:

In [4]:
from datetime import datetime

In [5]:
start = datetime.now()
[largest_prime_factor(i) for i in range(500_000)]
end = datetime.now()
print("main proc: ", end-start)

main proc:  0:00:08.006836


Now we do the same using ipyparallel

In [ ]:
start = datetime.now()
with ipp.Cluster() as rc:  # remote computer
    view = rc.load_balanced_view()

    work_begins = datetime.now()

    async_result = view.map_async(largest_prime_factor, list(range(500_000)))

    async_result.wait_interactive()

    result = async_result.get()

    end = datetime.now()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

In [ ]:
print("ipyparallel", end-start)
print("     warmup", work_begins-start)
print("  real work", end-work_begins)


Now let's try from julia

In [11]:
try:
    import julia
except ImportError:
    %pip install julia
    import julia
    print("this might take a while...",end="")
    julia.install()
    print("...done")

In [12]:
julia_code = """
function largest_prime_factor(n)
    p = n
    i = 2
    while i * i <= n
        if n % i != 0
            i += 1
        else
            n //= i
        end
    end
    return p, n
end
"""
with open('temp.jl', 'w') as fo:
    fo.write(julia_code)

In [13]:
j = julia.Julia()

UnsupportedPythonError: It seems your Julia and PyJulia setup are not supported.

Julia executable:
    julia
Python interpreter and libpython used by PyCall.jl:
    None
    None
Python interpreter used to import PyJulia and its libpython.
    C:\Users\madsenbj\Anaconda3\envs\github-pages\python.exe
    C:\Users\madsenbj\Anaconda3\envs\github-pages\python39.dll

In Julia >= 0.7, above two paths to `libpython` have to match exactly
in order for PyJulia to work out-of-the-box.  To configure PyCall.jl to use
Python interpreter "C:\Users\madsenbj\Anaconda3\envs\github-pages\python.exe",
run the following code in the Python REPL:

    >>> import julia
    >>> julia.install()

For more information, see:

    https://pyjulia.readthedocs.io/en/latest/troubleshooting.html


In [ ]:
x = j.include("temp.jl")